<a href="https://colab.research.google.com/github/AakashAhuja30/Pytorch/blob/main/Recurrent_NeuralNetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transform
import tqdm

In [2]:
#Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
#Hyperparameters
learning_rate = 0.001
batch_size=64
num_epochs=5
input_size=28
sequence_length=28
num_layers=2
hidden_size=256
num_classes=10

In [4]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers,num_classes):
    super(RNN,self).__init__()
    self.hidden_size= hidden_size
    self.num_layers = num_layers
    #Better performance with GRU
    self.rnn = nn.GRU(input_size,hidden_size,num_layers,batch_first=True)
    self.fc = nn.Linear(hidden_size*sequence_length,num_classes)

  def forward(self,x):
    h0 = torch.zeros(self.num_layers,x.size(0),self.hidden_size).to(device)

    #Forward prop
    out, _ = self.rnn(x,h0)
    out = out.reshape(out.shape[0],-1)
    out = self.fc(out)
    return out

In [6]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers,num_classes):
    super(LSTM,self).__init__()
    self.hidden_size= hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size,hidden_size,num_layers,batch_first=True)
    self.fc = nn.Linear(hidden_size*sequence_length,num_classes)

  def forward(self,x):
    h0 = torch.zeros(self.num_layers,x.size(0),self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers,x.size(0),self.hidden_size).to(device)

    #Forward prop
    out, _ = self.lstm(x,(h0,c0))
    out = out.reshape(out.shape[0],-1)
    out = self.fc(out)
    return out

In [7]:
#Load Data
train_dataset = datasets.MNIST(root='dataset/',train = True, transform=transform.ToTensor(), download=True)
train_loader = DataLoader(dataset = train_dataset , batch_size=batch_size,shuffle=True)
test_dataset = datasets.MNIST(root='dataset/',train = False, transform=transform.ToTensor(), download=True)
test_loader = DataLoader(dataset = test_dataset , batch_size=batch_size,shuffle=True)

In [7]:
#Initialize Network
model= RNN(input_size,hidden_size,num_layers, num_classes).to(device)
#Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [8]:
examples = iter(train_loader)
samples, labels = examples.next()
print(samples.shape, labels.shape)

torch.Size([64, 1, 28, 28]) torch.Size([64])


In [11]:
def check_accuracy(loader,model):
  if loader.dataset.train:
    print('Checking accuracy on Train data')
  else:
    print('Checking accuracy on Test data')

  num_samples =  0
  num_correct = 0
  model.eval()

  with torch.no_grad():
    for x,y in loader:
      x = x.to(device = device).squeeze(1)
      y = y.to(device = device)
      #x = x.view(x.shape[0],-1)

      scores= model(x)
      _,pred=torch.max(scores,1)
      num_correct += (pred ==y).sum()
      num_samples += pred.size(0)

    print(f'Accuracy is :{(num_correct/num_samples)*100}')

  model.train()  
  

In [10]:
for epoch in range(num_epochs):
  for batch_idx, (data,target) in enumerate(train_loader):
    data = data.to(device = device).squeeze(1)
    target = target.to(device = device)
    
    #Forward pass
    output = model(data)
    loss = criterion(output,target)

    #Backward pass
    optimizer.zero_grad()
    loss.backward()

    #Gradient descent
    optimizer.step()
    
  print(f'epoch{epoch+1}:')
  check_accuracy(train_loader,model)
  check_accuracy(test_loader,model)

epoch1:
Checking accuracy on Train data
Accuracy is :97.73999786376953
Checking accuracy on Test data
Accuracy is :97.64999389648438
epoch2:
Checking accuracy on Train data
Accuracy is :98.86833190917969
Checking accuracy on Test data
Accuracy is :98.68999481201172
epoch3:
Checking accuracy on Train data
Accuracy is :98.81500244140625
Checking accuracy on Test data
Accuracy is :98.5999984741211
epoch4:
Checking accuracy on Train data
Accuracy is :99.05333709716797
Checking accuracy on Test data
Accuracy is :98.65999603271484
epoch5:
Checking accuracy on Train data
Accuracy is :98.98666381835938
Checking accuracy on Test data
Accuracy is :98.40999603271484
epoch6:
Checking accuracy on Train data
Accuracy is :99.44499969482422
Checking accuracy on Test data
Accuracy is :98.82999420166016
epoch7:
Checking accuracy on Train data
Accuracy is :99.23999786376953
Checking accuracy on Test data
Accuracy is :98.7199935913086
epoch8:
Checking accuracy on Train data
Accuracy is :99.27166748046875


In [9]:
#Initialize Network
model= LSTM(input_size,hidden_size,num_layers, num_classes).to(device)
#Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [12]:
for epoch in range(num_epochs):
  for batch_idx, (data,target) in enumerate(train_loader):
    data = data.to(device = device).squeeze(1)
    target = target.to(device = device)
    
    #Forward pass
    output = model(data)
    loss = criterion(output,target)

    #Backward pass
    optimizer.zero_grad()
    loss.backward()

    #Gradient descent
    optimizer.step()
    
  print(f'epoch{epoch+1}:')
  check_accuracy(train_loader,model)
  check_accuracy(test_loader,model)

epoch1:
Checking accuracy on Train data
Accuracy is :98.67832946777344
Checking accuracy on Test data
Accuracy is :98.62999725341797
epoch2:
Checking accuracy on Train data
Accuracy is :98.95500183105469
Checking accuracy on Test data
Accuracy is :98.70999145507812
epoch3:
Checking accuracy on Train data
Accuracy is :99.2066650390625
Checking accuracy on Test data
Accuracy is :98.80999755859375
epoch4:
Checking accuracy on Train data
Accuracy is :99.1883316040039
Checking accuracy on Test data
Accuracy is :98.75999450683594
epoch5:
Checking accuracy on Train data
Accuracy is :99.42333984375
Checking accuracy on Test data
Accuracy is :99.04999542236328
